In [2]:
import pandas as pd
import pitcher

In [5]:
#currently these functions will only grab pitches CU, FF, SL, and CH so only use verlander and scherzer datasets
player = 'scherzer'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = pitcher.get_reps(ABs)
reps = pitcher.drop_nas(reps)

In [6]:
data = []
for rep in reps:
    row = []
    prev_pitches,prev_types,pre_pitch,ptype = rep
    prevs = []
    for prev in prev_pitches:
        prevs += prev
    row += prevs + pre_pitch + [ptype]
    data.append(row)

Each row of data now consists of:
- three previous pitch vectors (length 9) of at-bat, if the pitch-to-be-predicted is the first, second, or third pitch the at-bat, the previous empty spots will be filled with zeroes.
- one game state vector (length 15) representing the game state of the pitch-to-be-predicted
- the pitch-to-be-predicted 'NF' or 'FF', not-fastball or fourseam-fastball

running the cell above to get 'data' gives a list of rows each of length 43, which represents the 3 length 9 vectors plus the one length 15 vector plus the pitch type, i.e. the inputs to the network.